<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Onshape_API_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. READ ME - Onshape API Snippets Overview
This snippets library is split into numbered sections to help you get started with interacting with Onshape's REST API using their Python Client.

**[Click here](https://cad.onshape.com/glassworks/#/) to see the full OpenAPI documentation of the Onshape REST API**

***Start by using snippet 0.1*** to import the client and configuring you connection with API Keys generated in [Onshape's Dev Portal](https://dev-portal.onshape.com/). You can connect the snippets to your Onshape document by editing the "did", "eid", and "wid" from the URL of your Onshape document following the pattern https://cad.onshape.com/documents/'did'/w/'wid'/e/'eid'. Snippets are organized by API endpoints below.
00. Connect (start here)
01. PartStudios
02. Parts
03. MetaData
04. FeatureStudios
05. Elements
06. AppElements
07. Assemblies
08. Thumbnails
09. Documents
10. Users
11. Comments

All "GET" snippets reference [this public Onshape document](https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340). The default parameters for the did, wid, and eid will only work for "GET" requests because your API keys will not be able to edit the public document with a "POST" request.

# 00.1 Connect - Import and Configure Client

In [ ]:
#@title Import and Setup Onshape Client

!pip install onshape-client
from onshape_client.client import Client
from onshape_client.onshape_url import OnshapeElement
import json

#@markdown Chage the base if using an enterprise (i.e. "https://ptc.onshape.com")
base = 'https://cad.onshape.com' #@param {type:"string"}

#@markdown Would you like to import your API keys from a file, or copy and paste them directly?
keyImportOption = "Upload Keys from File" #@param ["Upload Keys from File", "Copy/Paste Keys"]

from IPython.display import clear_output 
clear_output()
print("Onshape Client successfully imported!")

if keyImportOption == "Upload Keys from File":
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    execfile(fn)

  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')
else:
  access = input("Paste your Onshape Access Key: ")
  secret = input("Paste your Onshape Secret Key: ")
  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')


# 00.1.1 Connect - Import Python Client
This snippet shows how to install the [python-client from Onshape](https://github.com/onshape-public/onshape-clients/tree/master/python)

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
from onshape_client.onshape_url import OnshapeElement
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

# 00.2 Connect - Configure with API Keys in separate document (recommended)
This snippet shows how you can pull API keys from a document stored in your google drive. **Good for sharing a notebook but keeping your keys secret**. Run the first snippet to load files from drive, then upload a python file that includes your access and secret keys as strings to your drive (can upload from Colab with the folder icon on the left menu).

The colabkeys.py file should look something like the following (below are fake keys - replace with your own from [dev-portal.onshape.com](https://dev-portal.onshape.com/)):
```
access = 'Z97asb1257FkEwzp3EmCpa'
secret = 'nHwN3q7asdvg4aryFC9rxYyo9U4o415WokYa8VOQ9YfUh4Zx'
```


In [ ]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  execfile(fn)

client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

# 00.2.1 Connect - Import keys from Google Drive file

If your "colabkeys.py" file is saved in your google drive, you'll need to run the code cell below to allow Colab to access drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
def include(filename):
  if os.path.exists(filename): 
    execfile(filename)
include('/content/drive/MyDrive/colabkeys.py')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

# 00.2.2 Connect - Local runtime
First [follow these instructions](https://research.google.com/colaboratory/local-runtimes.html) for connecting to a local runtime, then run the snippet below to find your local file with your api keys.

In [ ]:
#@title Import and Setup Onshape Client
#@markdown First connect to a local runtime following <a href="https://research.google.com/colaboratory/local-runtimes.html" target="_blank">these instrucitons</a>

!pip install onshape-client
from onshape_client.client import Client
from onshape_client.onshape_url import OnshapeElement
import json

#@markdown Chage the base if using an enterprise (i.e. "https://ptc.onshape.com")
base = 'https://cad.onshape.com' #@param {type:"string"}

#@markdown Would you like to import your API keys from a file, or copy and paste them directly?
keyImportOption = "Upload Keys from File" #@param ["Upload Keys from File", "Copy/Paste Keys"]

from IPython.display import clear_output 
clear_output()
print("Onshape Client successfully imported!")

if keyImportOption == "Upload Keys from File":
  filePath = input("Input the filepath of the python file with your API Keys (e.g. './Documents/apikeys.py'): ")
  exec(open(filePath).read())
  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')
else:
  access = input("Paste your Onshape Access Key: ")
  secret = input("Paste your Onshape Secret Key: ")
  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')


# 01.1 PartStudios - Get Mass Properties
For certain API endpoints, you must pass specific endpoints. For this example, you need to provide the element id of a part studio to get the mass properties for the parts in that part studio.

In [ ]:
#@title Get Mass Properties of Parts in a Part Studio
#@markdown Defines funciton `getMassProp(url: str)`, which returns JSON of mass properties for all parts in a part studio
url = 'https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340' #@param {type:"string"}
showResponse = False #@param {type:"boolean"}

def getMassProp(url: str):
  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'
  element = OnshapeElement(url)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed
if showResponse:
  massProp = getMassProp(url)
  print(json.dumps(massProp, indent=4, sort_keys=True))
else:
  pass

# 01.1.1 PartStudios - Get Mass Properties and Calculate Density
Sample of using the mass properties endpoint to get information and do calculations on the data. Sample document here: https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))
# parse json to get mass and volume for all parts in the document
volume = json.loads(response.data)['bodies']['-all-']['volume'][0] # three element array is volume, max volume, and minimum volume
mass = json.loads(response.data)['bodies']['-all-']['mass'][0] # three element array is mass, maximum mass, and minimum mass

# calculate density with mass and volume in kg/m^3
density = mass/volume
print(density)

# 01.1.2 PartStudios - Get Mass Properties from Configured Part
In this example, the document has configuration variables set up to drive parameters of the geometry. We change the "querry parameters" of the request to get the mass property for the different values.

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'

method = 'GET'

# add configureation to the params of the request
params = {'configuration': 'Length%3D0.2%2Bmeter'}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))

# 01.1.3 PartStudios - Get Mass Prop for series of configurations and plot

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'GET'
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}
params = {}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(json.loads(response.data))
x = []
y = []

for i in range(1,5):
  x.append(i)
  configdef = 'Length%3D'+str(i)+'%2Bmeter'+';'+'Width%3D'+str(i)+'%2Bmeter'
  params = {'configuration': configdef}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  massProp = json.loads(response.data)
  y.append(massProp['bodies']['-all-']['inertia'][0])

  # The command below prints the entire JSON response from Onshape
  print(massProp['bodies']['-all-']['inertia'][0])
plt.plot(x, y, 'o', color='black');


# 01.2 PartStudios - Make Part Studio
This is an example of a post request that creates a new part studio in an Onshape document.

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid'

did = '4bda16c648566259ea1b4e4c'
wid = 'c299b9fc994574c2637e871d'

method = 'POST'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))

# 01.3 PartStudios - Get Feature List

In [ ]:
#@title Get Feature List from Part Studio
#@markdown Defines funciton `getFeatureList(url: str)`, which returns JSON of all features in a part studio
url = 'https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340' #@param {type:"string"}
showResponse = False #@param {type:"boolean"}
def getFeatureList(url: str):
  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features'

  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'  
  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed
if showResponse:
  featureResponse = getFeatureList(url)
  print(json.dumps(featureResponse, indent=4, sort_keys=True))
else:
  pass


# 01.4 PartStudios - Update Feature

First get the feature, then update the geometry, then send it back


In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features'

#https://cad.onshape.com/documents/bb1a0fa7536a74f282879c0a/w/107d615fca7419084dd2ddf3/e/1cb13d9abf5bab6bcb8e3668
did = 'bb1a0fa7536a74f282879c0a'
wid = '107d615fca7419084dd2ddf3'
eid = '872dc4906d7f740d74474a8c'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'GET'
params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
# print(json.loads(response.data)["features"][0]["message"]["entities"][0]["message"]["geometry"]["message"])

newFeature = json.loads(response.data)["features"][0]
newFeature["message"]["entities"][0]["message"]["geometry"]["message"]["radius"] = 0.01
print(newFeature["message"]["entities"][0]["message"]["geometry"]["message"])

print(json.loads(response.data)["sourceMicroversion"])
print(json.loads(response.data)["serializationVersion"])

fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features/featureid/fid'

fid = json.loads(response.data)["features"][0]["message"]["featureId"]

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid, 1)
fixed_url = fixed_url.replace('fid', fid)

print(fixed_url)

method = 'POST'
params = {}
payload = {'feature': newFeature,
           'serializationVersion': json.loads(response.data)["serializationVersion"],
           'sourceMicroversion': json.loads(response.data)["sourceMicroversion"]}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# 01.5 PartStudios - Export STL

In [ ]:
def exportSTL(url: str, filename="OnshapePart.stl"):
  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/stl'
  element = OnshapeElement(url)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+octet-stream',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  with open(filename, 'wb') as f:
    f.write(response.data.encode())

exportSTL('https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340',"colabDemo.stl")

#01.6 PartStudios - Get Shaded View

In [ ]:
#@title Get Shaded View of PartSutdio
#@markdown Defines funciton `partStudioShadedView(url:str,viewMatrix = "front",pixelSize = 0.003)`, which returns the base64 image string of a shaded view of a part studio

#@markdown URL must be to an PartStudio
url = 'https://cad.onshape.com/documents/1e809243865e8d5244fcd096/w/d031ceaee074757da46d14b9/e/e370ef8f9ad617d9ae83f494' #@param {type:"string"}
#@markdown viewMatrix can be any face direction or isometric as a string, or a 1x12 view matrix
viewMatrix = "isometric" #@param {type:"string"}
#@markdown pixelSize is the size in meters for each pixel. If 0, it will fill the image size output
pixelSize = 0.0005 #@param {type:"number"}
showImage = False #@param {type:"boolean"}

from IPython.display import Image
import base64

def partStudioShadedView(url:str,viewMatrix = "front",pixelSize = 0.003):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/shadedviews'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  if any(face in viewMatrix for face in ["front","back","top","bottom","left","right"]):
    matrix = viewMatrix
  elif viewMatrix == "isometric":
    matrix = "0.612,0.612,0,0,-0.354,0.354,0.707,0,0.707,-0.707,0.707,0"
  elif isinstance(viewMatrix,list):
    matrix = str(matrix).replace('[','').replace(']','')

  ## View Matrix below is roughly isometric
  params = {'viewMatrix':matrix,
            'edges':'show',
            'outputHeight':600,
            'outputWidth':1000,
            'pixelSize':pixelSize}
  # print(params)
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
              'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  imgdata = base64.b64decode(parsed['images'][0])
  with open("image.jpg", 'wb') as f:
    f.write(imgdata)
  
  return imgdata

img = partStudioShadedView(url,viewMatrix,pixelSize)
if showImage:
  display(Image(img))
else:
  pass

# 02.1 Parts - Get All Parts in Document
This snippet shows a sample of getting all of the parts in an Onshape document. 

In [ ]:
#@title Get All Parts in Document
#@markdown Funciton `getPartsInDocument(url: str)` returns JSON of all parts in a document
url = 'https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}
def getPartsInDocument(url: str):
  fixed_url = '/api/parts/d/did/w/wid'

  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

if showResponse:
  partResponse = getPartsInDocument(url)
  print(json.dumps(partResponse, indent=4, sort_keys=True))
else:
  pass

# 02.2 Parts - Get All Parts in Part Studio

In [ ]:
#@title Get Parts in Part Studio
#@markdown Funciton `getPartsInPartStudio(url: str)` returns JSON of all parts in a part studio
url = 'https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}
def getPartsInPartStudio(url: str):
  fixed_url = '/api/parts/d/did/w/wid/e/eid/'

  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

if showResponse:
  partResponse = getPartsInPartStudio('https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340')
  print(json.dumps(partResponse, indent=4, sort_keys=True))
else:
  pass

# 03.1 MetaData - Get Element MetaData

In [ ]:
fixed_url = '/api/metadata/d/did/w/wid/e/eid'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
for i in json.loads(response.data)['properties']:
  print(i['name'])
  print(i['value'])

# 03.2 MetaData - Get Part MetaData
You will need a Part ID for this API endpoint. You can get Part ID's with snippet 2.1 above.

In [ ]:
fixed_url = '/api/metadata/d/did/w/wid/e/eid/p/pid'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'
pid = 'JHD'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)
fixed_url = fixed_url.replace('pid', pid)

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

for i in json.loads(response.data)['properties']:
  print(i['name'])
  print(i['value'])

# 03.2.1 MetaData - Update Part Color
The function below gets the metadata for a part, then changes the part's color and opacity, and sends a post request to update it. **The function needs a part id (pid) and element id (eid) which can be found with the snippet in 2.1 above**.

In [ ]:
#@title Update Part Color
#@markdown Defines funciton `updateColor(url,eid,pid,newOpacity,newColors)`, which updates the color of the specified part to the specified values

#@markdown URL can be to your assembly
url = 'https://cad.onshape.com/documents/fb00ed566c3e0934ae91d74c/w/80749fc9b1aa0adab776dc56/e/f17ce18736063f03b8d7dd9a' #@param {type:"string"}
#@markdown eid is the element ID of the partStudio that has the part you want to change the color of
eid = "d57f24d933d2521fce153c7d" #@param {type:"string"}
#@markdown pid is the part ID of the part you want to change the color of
pid = "JHD" #@param {type:"string"}
#@markdown newOpacity is a value from 0-255, or input -1 to keep opacity the same
newOpacity = 5 #@param {type:"number"}
#@markdown newColor is a JSON object of form `{'red': 10, 'green': 255, 'blue': 10}`, or input "none" to keep color the same
newColors = "none" #@param
showResponse = False #@param {type:"boolean"}

def updateColor(url,eid,pid,newOpacity,newColors):

  fixed_url = '/api/metadata/d/did/w/wid/e/eid/p/pid'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', eid)
  fixed_url = fixed_url.replace('pid', pid)

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}  

  response = client.api_client.request('GET', url=base + fixed_url,
  query_params=params, headers=headers, body=payload)

  partMetaData = json.loads(response.data)
  partProperties = partMetaData['properties']

  # # Print current appearance values
  # for x in partProperties:
  #   if x['name'] == "Appearance":
  #     partAppearance = x
  #     break
  # print(partAppearance['value'])

  for i in range(len(partProperties)):
    if partProperties[i]['name'] == "Appearance":
      if newOpacity == -1:
        pass
      else:
        partProperties[i]['value']['opacity'] = newOpacity
      if newColors == "none":
        pass
      else:
        partProperties[i]['value']['color'] = newColors
      break

  partMetaData['properties'] = partProperties
  payload = partMetaData

  response = client.api_client.request('POST', url=base + fixed_url,
  query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  return parsed

updatedColor = updateColor(url,eid,pid,newOpacity,newColors)
if showResponse:
  print(json.dumps(updatedColor, indent=4, sort_keys=True))
else:
  pass

# 04.1 FeatureStudios - Create Feature Studio
First snippet creates a FeatureStudio

In [ ]:
def addFeatureStudio(url:str,name="ColabFeature"):
  fixed_url = '/api/featurestudios/d/did/w/wid'

  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)

  method = 'POST'

  params = {}
  payload = {"name":name}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  # The command below prints the entire JSON response from Onshape
  print(json.loads(response.data))

addFeatureStudio('https://cad.onshape.com/documents/7820391e3a9b37485d78da07/w/62c67fd520278fbfa3ebd529/e/c83cc398c91add81154089d7')

# 04.2 FeatureStudios - Get FeatureSpecs
Element ID must be for a FeatureStudio

In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid/featurespecs'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/7a1ceda85403a0d972055ba7
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = '7a1ceda85403a0d972055ba7'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(response.data)

# 04.3 FeatureStudios - Get Feature Content

In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/7a1ceda85403a0d972055ba7
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = '7a1ceda85403a0d972055ba7'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(response.data)

# 04.4 FeatureStudios - Update Feature Content


In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid'

did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'
eid = '7e843682a5c2ec5c8a0bb1af'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

newBody = json.loads(response.data)
print(newBody["contents"]) #print current contents of featurestudio

#define new feature here
feature = """
FeatureScript 1511;
import(path : "onshape/std/geometry.fs", version : "1511.0");

annotation { "Feature Type Name" : "My Feature" }
export const myFeature = defineFeature(function(context is Context, id is Id, definition is map)
    precondition
    {
        // Define the parameters of the feature type
    }
    {
        // Define the function's action
        fCuboid(context, id + "cuboid1", {
                "corner1" : vector(0, 0, 0) * inch,
                "corner2" : vector(1, 1, 1) * inch
        });
    });
"""
newBody["contents"] = feature
payload = newBody

method = 'POST'
response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# 05.1 Elements - Get Configurations

In [ ]:
#@title Get List of Configurations from an Element
#@markdown Defines funciton `getElementConfiguration(url: str)`, which returns JSON of all configurations for an element

url = 'https://cad.onshape.com/documents/eecccfab1c8776ea7391b956/w/31f74da518761683b404d0a1/e/5d75c81872788ee207559d27' #@param {type:"string"}
showResponse = False #@param {type:"boolean"}
def getElementConfiguration(url: str):
  fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
          'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed
if showResponse:
  elementConfiguration = getElementConfiguration(url)
  print(json.dumps(elementConfiguration, indent=4, sort_keys=True))
else:
  pass

# 06.1 AppElements - Create AppElement

In [ ]:
#@title Add App Element to a document
#@markdown Defines funciton `addAppElement(url: str,name="Colab AppElement")`, which adds an app element to a document with a specified name and returns the JSON response

url = 'https://cad.onshape.com/documents/1619b643b35c8e9c2ef677c8/w/4df7c19535def518bee858f1/e/d742899ec15fd06180470229' #@param {type:"string"}
name = "Octoprint AppElement" #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def addAppElement(url:str,name="Colab AppElement"):
  fixed_url = '/api/appelements/d/did/w/wid'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)

  method = 'POST'

  params = {}
  payload = {
        "formatId": "com.python",
        "name": name,
        "description": "Created App Element",
        "jsonTree": {"hello":"world"}
      }
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

if showResponse:
  appElement = addAppElement(url, name)
  print(json.dumps(appElement, indent=4, sort_keys=True))
else:
  pass

# 06.2 AppElements - Get JSON Tree Data

Element ID must be an application element

In [ ]:
#@title Get JSON tree from App Element
#@markdown Defines funciton `getJsonTree(url: str,name="Colab AppElement")`, which adds an app element to a document with a specified name and returns the JSON response

url = 'https://cad.onshape.com/documents/7820391e3a9b37485d78da07/w/62c67fd520278fbfa3ebd529/e/71e0929b813fbc1b08c92c99' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def getJsonTree(url:str):
  fixed_url = '/api/appelements/d/did/w/wid/e/eid/content/json'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)

  return parsed

if showResponse:
  jsonTree = getJsonTree(url)
  print(json.dumps(jsonTree, indent=4, sort_keys=True))
else:
  pass

# 06.3 AppElements - Update JSON Tree

Change value of existing JSON key

In [ ]:
#@title Update JSON tree in App Element
#@markdown Defines funciton `updateAppElementJsonKey(url:str,keyName="foo",keyValue="bar")`, which adds an app element to a document with a specified name and returns the JSON response

#@markdown **Note: you must define `getJsonTree()` function before using this**
url = 'https://cad.onshape.com/documents/7820391e3a9b37485d78da07/w/62c67fd520278fbfa3ebd529/e/71e0929b813fbc1b08c92c99' #@param {type:"string"}
keyName = 'hello' #@param {type:"string"}
keyValue = 'bye' #@param {type:"string"}

showResponse = True #@param {type:"boolean"}

def updateAppElementJsonKey(url:str,keyName="foo",keyValue="bar"):
  fixed_url = '/api/appelements/d/did/w/wid/e/eid/content'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'POST'

  params = {}
  payload = {
      "parentChangeId": getJsonTree(url)['changeId'],
      "jsonTreeEdit": {'btType' : "BTJEditChange-2636", 
                      'path' : { 'btType' : 'BTJPath-3073', 'startNode' : '', 'path' : [{ 'btType' : 'BTJPathKey-3221', 'key' : keyName }] }, 
                      'value' : keyValue }
      }

  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

if showResponse:
  newJsonTree = updateAppElementJsonKey(url,keyName,keyValue)
  print(json.dumps(newJsonTree, indent=4, sort_keys=True))
else:
  pass

# 06.3.1 AppElements - Update JSON Tree

Add new json key

In [ ]:
#@title Add new key to JSON tree in App Element
#@markdown Defines funciton `addAppElementJsonKey(url:str,newKeyName="foo",newKeyValue="bar")`, which adds an app element to a document with a specified name and returns the JSON response

#@markdown **Note: you must define `getJsonTree()` function before using this**
url = 'https://cad.onshape.com/documents/7820391e3a9b37485d78da07/w/62c67fd520278fbfa3ebd529/e/71e0929b813fbc1b08c92c99' #@param {type:"string"}
newKeyName = 'foo' #@param {type:"string"}
newKeyValue = 'bar' #@param {type:"string"}

showResponse = True #@param {type:"boolean"}

def addAppElementJsonKey(url:str,newKeyName="foo",newKeyValue="bar"):
  fixed_url = '/api/appelements/d/did/w/wid/e/eid/content'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'POST'

  # Insertion: { 'btType' : 'BTJEditInsert-2523', 'path' : path, 'value' : newValue }

  params = {}
  payload = {
      "parentChangeId": getJsonTree(url)['changeId'],
      "jsonTreeEdit": {'btType' : 'BTJEditInsert-2523', 
                      'path' : { 'btType' : 'BTJPath-3073', 'startNode' : '', 'path' : [{ 'btType' : 'BTJPathKey-3221', 'key' : newKeyName }] }, 
                      'value' : newKeyValue }
      }

  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

if showResponse:
  newJsonTree = addAppElementJsonKey(url,newKeyName,newKeyValue)
  print(json.dumps(newJsonTree, indent=4, sort_keys=True))
else:
  pass

# 06.4 AppElements - Get JSON Tree History

In [ ]:
def getJsonTreeHistory():
  fixed_url = '/api/appelements/d/did/w/wid/e/eid/content/history'

  # https://cad.onshape.com/documents/9e0878dfcce12a4a996be8a7/w/0aab7305e4b4f0d3537962d6/e/606b7bd0a144819c887bd396
  did = '9e0878dfcce12a4a996be8a7'
  wid = '0aab7305e4b4f0d3537962d6'
  eid = '55015109301f6462ffe9a058'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  print(base + fixed_url)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

print(getJsonTreeHistory())

# 07.1 Assemblies - Get Assembly Definition

In [ ]:
#@title Get Assembly Definition
#@markdown Function `getAssemblyDefinition(url: str)` returns JSON of all parts in a document. The URL can be any element from the document.
url = 'https://cad.onshape.com/documents/bb22099573adad2eb4ea7daa/w/7c08e43470d8f47139b916ab/e/67cb1d22c65fb548487543b6' #@param {type:"string"}
showResponse = False#@param {type:"boolean"}
listParts = True#@param {type:"boolean"}
def getAssemblyDefinition(url: str):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

assemblyDefinitionResponse = getAssemblyDefinition(url)

if showResponse:
  print(json.dumps(assemblyDefinitionResponse, indent=4, sort_keys=True))
if listParts:
  for instance in assemblyDefinitionResponse['rootAssembly']['instances']:
    name = instance['name']
    id = instance['id']
    print(f'Part named {name} has id {id}')
else:
  pass

# 07.2 Assemblies - Create Assembly Instance
The snippet below shows how to add all Parts from a Part Studio into an Assembly within the same document. If you want to pull a part from a different document, you must specify a versionID for the source document

In [ ]:
# Specify the document and element ID of the source part you want to insert
sourceDid = '062ac5b9f4d2e8b873653e76'
sourceEid = 'b508a03a28dd5add1fc6e5c0'

# Uncomment section below and use only if sourceDid is different than the assembly Did

# # First get the version of the document you want to pull from
# fixed_url = '/api/documents/d/did/versions'
# fixed_url = fixed_url.replace('did', sourceDid)

# method = 'GET'

# params = {}
# payload = {}
# headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
#            'Content-Type': 'application/json'}

# response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
# version = json.loads(response.data)[0]['id']


# Then set up POST request to assembly you want to add to

fixed_url = '/api/assemblies/d/did/w/wid/e/eid/instances'

# Assembly you want to add instance to
# https://cad.onshape.com/documents/062ac5b9f4d2e8b873653e76/w/4c2c6371b76c676921ae8032/e/0a9a4d0604f036fd608e5120
did = '062ac5b9f4d2e8b873653e76'
wid = '4c2c6371b76c676921ae8032'
eid = '0a9a4d0604f036fd608e5120'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'POST'

params = {}
payload = {'documentId': sourceDid,
           'elementId': sourceEid,
          #  'versionId':version, # only include versionID if source part is in a different document than the assembly
           'isWholePartStudio':True,
          #  'configuration':'MotorShape=Default' # must specify configuration if part is configured
           }

headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

print(response.status)

# 07.2.1 Assemblies - Create Instance from Assembly
This snippet shows how to add a sub-assembly from a linked document into a new assembly

In [ ]:
# First get the version of the document you want to pull from
fixed_url = '/api/documents/d/did/versions'
did = 'a1b859a8fa32ed77b6a98ef0'
fixed_url = fixed_url.replace('did', did)

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
version = json.loads(response.data)[0]['id']


# Then set up POST request to assembly you want to add to

fixed_url = '/api/assemblies/d/did/w/wid/e/eid/instances'

# Assembly you want to add instance to
# https://cad.onshape.com/documents/f20e3b39b4d19572ee26665d/w/26800f92770dbe88d66129fe/e/ff736ca1cb4b923812bf7f36
did = 'f20e3b39b4d19572ee26665d'
wid = '26800f92770dbe88d66129fe'
eid = 'ff736ca1cb4b923812bf7f36'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'POST'

# specify the document ID and element ID for the assmelby you'd like to add
checkptDid = 'a1b859a8fa32ed77b6a98ef0'
checkptEid = '494c89e6e3215f70111cd3c2'

params = {}
payload = {'documentId': checkptDid,
           'elementId': checkptEid,
           'versionId':version,
           'isAssembly':True}

headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

print(response)

# 07.3 Assemblies - Occurrence Transforms

In [ ]:
#@title Run this cell to load matrix helper functions

import math
import numpy

def IdentitySixteen():
  m = [
      1, 0, 0, 0,
      0, 1, 0, 0,
      0, 0, 1, 0,
      0, 0, 0, 1
      ]
  return m

def DiceTransform(value):
    switcher = {
        0: [
            0, 0, 1, 0,
            0, 1, 0, 0,
            -1, 0, 0, 0,
            0, 0, 0, 1
           ],
        1: [
            0, 0, 1, 0,
            0, 1, 0, 0,
            -1, 0, 0, 0,
            0, 0, 0, 1
           ],
        2: [
            1, 0, 0, 0,
            0, 0, 1, 0,
            0, -1, 0, 0,
            0, 0, 0, 1
           ],
        3: [
            1, 0, 0, 0,
            0, -1, 0, 0,
            0, 0, -1, 0,
            0, 0, 0, 1
           ],
        4: [
            1, 0, 0, 0,
            0, 1, 0, 0,
            0, 0, 1, 0,
            0, 0, 0, 1
           ],
        5: [
            1, 0, 0, 0,
            0, 0, -1, 0,
            0, 1, 0, 0,
            0, 0, 0, 1
           ],
        6: [
            0, 0, -1, 0,
            0, 1, 0, 0,
            1, 0, 0, 0,
            0, 0, 0, 1
           ] 
    }
  
    # get() method of dictionary data type returns 
    # value of passed argument if it is present 
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(value, "nothing")

def DiceMove(value,x1,y1):
  matrix = DiceTransform(value)
  matrix[3] = x1
  matrix[7] = y1
  return matrix

def DiceFollow(diceMatrix,tcpMatrix):
  diceMatrix[3] = tcpMatrix[3]
  diceMatrix[7] = tcpMatrix[7]
  diceMatrix[11] = tcpMatrix[11]
  return diceMatrix
  
def sixteenToFourByFour(matrix):
  fourbyfour = [[matrix[0],matrix[1],matrix[2],matrix[3]],
                [matrix[4],matrix[5],matrix[6],matrix[7]],
                [matrix[8],matrix[9],matrix[10],matrix[11]],
                [matrix[12],matrix[13],matrix[14],matrix[15]]]
  return fourbyfour

def fourByFourToSixteen(matrix):
  sixteen = [matrix[0][0],matrix[0][1],matrix[0][2],matrix[0][3],
             matrix[1][0],matrix[1][1],matrix[1][2],matrix[1][3],
             matrix[2][0],matrix[2][1],matrix[2][2],matrix[2][3],
             matrix[3][0],matrix[3][1],matrix[3][2],matrix[3][3]]
  return sixteen

def clockwiseSpinX(theta):
  m = [[1, 0, 0, 0],
       [0, math.cos(theta), math.sin(theta), 0],
       [0, -math.sin(theta), math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinY(theta):
  m = [[math.cos(theta), 0, math.sin(theta), 0],
       [0, 1, 0, 0],
       [-math.sin(theta), 0, math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinZ(theta):
  m = [[math.cos(theta), math.sin(theta), 0, 0],
       [-math.sin(theta), math.cos(theta), 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]]
  return m


In [ ]:
#@title Transformation Matrix
#@markdown Move the part with specified id. (Use getAssemblyDefinition function to get part IDs)
url = 'https://cad.onshape.com/documents/bb22099573adad2eb4ea7daa/w/7c08e43470d8f47139b916ab/e/67cb1d22c65fb548487543b6' #@param {type:"string"}
partIdToMove = 'Mqmjho+TUKGFqtaQi'#@param {type:'string'}
isRelative = False#@param {type:'boolean'}
#@markdown Translate object to x1,y1,z1 (if isRelative is False) or by x1,y1,z1 in the x,y,z directions (if isRelative is True)
x1 = 0.05#@param
y1 = 0.05#@param
z1 = 0#@param
#@markdown Specify rotations (in radians) about the X and Z axis
clockwiseRotationX = math.pi/4#@param
clockwiseRotationZ = math.pi/2#@param

occurrences = getAssemblyDefinition(url)['rootAssembly']['occurrences']

for x in occurrences:
    if x['path'][0] == partIdToMove:
      occurrence = x

## Reset the transformation matrix
occurrence['transform'] = IdentitySixteen()
occurrence['transform'][3] = x1
occurrence['transform'][7] = y1
occurrence['transform'][11] = z1

## Put translation first, rotation second to rotate about global coordinate system first
rotMat = numpy.matmul(clockwiseSpinX(clockwiseRotationX),clockwiseSpinZ(clockwiseRotationZ))
transformMat = numpy.matmul(sixteenToFourByFour(occurrence['transform']),rotMat)
print("Transformation matrix:")
for x in transformMat:
  print(x)

## Send assembly occurence transforms
fixed_url = '/api/assemblies/d/did/w/wid/e/eid/occurrencetransforms'

element = OnshapeElement(url)
fixed_url = fixed_url.replace('did', element.did)
fixed_url = fixed_url.replace('wid', element.wvmid)
fixed_url = fixed_url.replace('eid', element.eid)

method = 'POST'

params = {}
payload = {'isRelative':isRelative,
           'occurrences':[occurrence],
           'transform':fourByFourToSixteen(transformMat)
    }

headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)


# 07.4 Assemblies - Mate Values
The snippet below shows how to get the values of the mate positions in an assembly. You can do a post request with the same JSON body and different values to update the mate values. **Note:** if you try to update too many mate values with one call, the assembly may not solve properly.

In [ ]:
#@title Get Mates from Assembly
#@markdown Defines funciton `getAssemblyMates(url)`, which returns JSON of mates in an assembly
url = 'https://cad.onshape.com/documents/1e809243865e8d5244fcd096/w/d031ceaee074757da46d14b9/e/e370ef8f9ad617d9ae83f494' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def getAssemblyMates(url):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'
  element = OnshapeElement(url)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
              'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

if showResponse:
  mates = getAssemblyMates(url)
  print(json.dumps(mates, indent=4, sort_keys=True))
else:
  pass

# 07.5 Assemblies - Set Mate Value by Name

The snippet below is a function that first gets the mate values from an assembly, then updates one of the mate angles and removes the other (we don't want to update it), and sends a post request to update one mate value.

In [ ]:
#@title Set Mates in Assembly
#@markdown Defines funciton `setMateByName(url,mateName,position)`, which updates the mate with the specified name to the specified position

#@markdown URL must be to an assembly
url = 'https://cad.onshape.com/documents/1e809243865e8d5244fcd096/w/d031ceaee074757da46d14b9/e/e370ef8f9ad617d9ae83f494' #@param {type:"string"}
mateName = "Revolute 1" #@param {type:"string"}
position =  0#@param {type:"number"}
showResponse = True #@param {type:"boolean"}

def getAssemblyMates(url):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'
  element = OnshapeElement(url)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
              'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

def setMateByName(url,mateName,position):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'
  mates = getAssemblyMates(url)
  element = OnshapeElement(url)
  

  for values in mates['mateValues']:
    if values['mateName'] == mateName:
      newMate = values
  
  if newMate['jsonType'] == 'Revolute':
    newMate['rotationZ'] = position
  elif newMate['jsonType'] == 'Slider':
    newMate['translationZ'] = position
  else:
    print('failed')
  
  method = 'POST'

  params = {}
  payload = {'mateValues':[newMate]}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
              'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed

newMates = setMateByName(url,mateName,position)
if showResponse:
  print(json.dumps(newMates, indent=4, sort_keys=True))
else:
  pass

# 07.6 Assemblies - Get Shaded View

In [ ]:
#@title Get Shaded View of Assembly
#@markdown Defines funciton `assembliesShadedView(url:str,viewMatrix = "front",pixelSize = 0.003,filename = "image.jpg")`, which returns the base64 image data of an assembly and saves the image as a jpeg called "image.jpg"

#@markdown URL must be to an assembly
url = 'https://cad.onshape.com/documents/3d09feac1ef676fa98d31eb9/w/51c15a206d5b6d3f6f0b16a5/e/c459cd771e8e664d6ce40fab' #@param {type:"string"}
#@markdown viewMatrix can be any face direction or isometric as a string, or a 1x12 view matrix
viewMatrix = "isometric" #@param {type:"string"}
#@markdown pixelSize is the size in meters for each pixel. If 0, it will fill the image size output
pixelSize = 0.0005 #@param {type:"number"}
showImage = False #@param {type:"boolean"}

from IPython.display import Image
import base64

def assembliesShadedView(url:str,viewMatrix = "front",pixelSize = 0.003,filename = "image.jpg"):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/shadedviews'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  if any(face in viewMatrix for face in ["front","back","top","bottom","left","right"]):
    matrix = viewMatrix
  elif viewMatrix == "isometric":
    matrix = "0.612,0.612,0,0,-0.354,0.354,0.707,0,0.707,-0.707,0.707,0"
  elif isinstance(viewMatrix,list):
    matrix = str(viewMatrix).replace('[','').replace(']','')

  ## View Matrix below is roughly isometric
  params = {'viewMatrix':matrix,
            'edges':'show',
            'outputHeight':600,
            'outputWidth':1000,
            'pixelSize':pixelSize}
  # print(params)
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
              'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  imgdata = base64.b64decode(parsed['images'][0])
  with open(filename, 'wb') as f:
    f.write(imgdata)
  
  return imgdata

img = assembliesShadedView(url,viewMatrix,pixelSize)
if showImage:
  display(Image(img))
else:
  pass

# 08.1 Thumbnails - Get document Thumbnails
The snippet below returns the URL of the png image with the document thumbnails

In [ ]:
fixed_url = '/api/thumbnails/d/did'

# https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340
did = '263517311c2ad139d4eb57ca'
wid = 'b45057ae06777e0c28bca6c5'
eid = 'd316bcbc694c9dbb6555f340'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(response.data)

fullResponse = json.loads(response.data)

# 09.1 Documents - Get User's Documents

In [ ]:
def documents():
  fixed_url = '/api/documents'
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

documents()

# 09.2 Documents - Get Current Microversion

In [ ]:
def getCurrentMicroversion():
  fixed_url = '/api/documents/d/did/w/wid/currentmicroversion'

  # https://cad.onshape.com/documents/9e0878dfcce12a4a996be8a7/w/0aab7305e4b4f0d3537962d6/e/606b7bd0a144819c887bd396
  did = '9e0878dfcce12a4a996be8a7'
  wid = '0aab7305e4b4f0d3537962d6'
  eid = '55015109301f6462ffe9a058'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  print(base + fixed_url)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

print(getCurrentMicroversion()['microversion'])

# 09.3 Documents - Get Document History



In [ ]:
def getDocumentHistory(url):
  fixed_url = '/api/documents/d/did/w/wid/documenthistory'
  element = OnshapeElement(url)

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  # fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

print(getDocumentHistory('https://cad.onshape.com/documents/263517311c2ad139d4eb57ca/w/b45057ae06777e0c28bca6c5/e/d316bcbc694c9dbb6555f340'))

# 09.4 Documents - Get Elements List

In [ ]:
#@title Get List of Elements in Document
#@markdown Function `documentElementsList(url: str)` returns JSON of all elements in a document
url = 'https://cad.onshape.com/documents/7820391e3a9b37485d78da07/w/62c67fd520278fbfa3ebd529/e/c83cc398c91add81154089d7' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def documentElementsList(url: str):
  fixed_url = '/api/documents/d/did/w/wid/elements'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

if showResponse:
  elementList = documentElementsList(url)
  print(json.dumps(elementList, indent=4, sort_keys=True))
else:
  pass

# 09.5 Documents - Get Document Info

In [ ]:
#@title Get Document Info
#@markdown Function `documentElement(url: str)` returns JSON of all elements in a document
url = 'https://cad.onshape.com/documents/e83ac518fd184ff1fe94beda/w/0e658e2953c99c66acdf3c09/e/0db5e995acdfb94b0b9237f4' #@param {type:"string"}
showResponse = False #@param {type:"boolean"}

def documentElement(url: str):
  fixed_url = '/api/documents/did'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

if showResponse:
  documentInfo = documentElement(url)
  print(json.dumps(documentInfo, indent=4, sort_keys=True))
else:
  pass

# 10.1 Users - Get User Settings

In [ ]:
#@title Get User Settings
#@markdown Function `userSettings()` returns JSON of user settings
showResponse = True #@param {type:"boolean"}

def userSettings():
  fixed_url = '/api/users/settings'
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

if showResponse:
  settings = userSettings()
  print(json.dumps(settings, indent=4, sort_keys=True))
else:
  pass

# 11.1 Comments - Get All Comments

In [ ]:
#@title Get List of Comments in Document
#@markdown Function `getComments(url: str)` returns JSON of all comments in a document
url = 'https://cad.onshape.com/documents/3d09feac1ef676fa98d31eb9/w/51c15a206d5b6d3f6f0b16a5/e/94c3256991c2c583c2556f86' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def getComments(url: str):
  fixed_url = '/api/comments'
  element = OnshapeElement(url)
  method = 'GET'

  params = {'did':element.did,
            'wid':element.wvmid,
            'eid':element.eid}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))
  return parsed

if showResponse:
  comments = getComments(url)
  print(json.dumps(comments, indent=4, sort_keys=True))
else:
  pass